In [1]:
import subprocess

from flask import Flask, request, Response
import jsonpickle
import numpy as np
import cv2
import tensorflow as tf

from models.recognizer import FruitRecognizer

Using TensorFlow backend.


In [2]:
global graph
graph = tf.get_default_graph()

In [3]:
reco = FruitRecognizer(model_path="models\saved_models\model1_vgg16_architecture.json",
                        weights_path="models\saved_models\model1_vgg16_best1_weights.hdf5")

In [7]:
app = Flask(__name__)

# route http posts to this method
@app.route('/api/recognize_fruit', methods=['POST'])
def test():
    r = request
    # convert string of image data to uint8
    nparr = np.fromstring(r.data, np.uint8)

    # decode image
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    # do some fancy processing here.... 
    with graph.as_default():
        res = reco.predict(img)

    # build a response dict to send back to client
    response = {'message': 'Your fruit is {}'.format(res)
                }
    # encode response using jsonpickle
    response_pickled = jsonpickle.encode(response)

    return Response(response=response_pickled, status=200, mimetype="application/json")

In [8]:
# start flask app
app.run(host="0.0.0.0", port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


In [11]:
import subprocess
import os

from flask import Flask, request, Response
import jsonpickle
import numpy as np
import cv2
import tensorflow as tf

from models.recognizer import FruitRecognizer
from RequestValidator import getImageByteArray

#base_dir = os.path.abspath(os.path.dirname(__file__))
model_path = os.path.join('models', "saved_models",
                          "model1_vgg16_architecture.json")
weights_path = os.path.join('models', "saved_models",
                            "model1_vgg16_best1_weights.hdf5")

global graph
graph = tf.get_default_graph()

version = subprocess.check_output(["git", "describe"]).strip()
print(version)

reco = FruitRecognizer(model_path=model_path,
                       weights_path=weights_path)

app = Flask(__name__)


# route http posts to this method
@app.route('/api/recognize', methods=['POST'])
def test():

    fileReceived = getImageByteArray(request.files)

    if not (fileReceived[0]):
        response = {'message': 'No file received'}
        response_pickled = jsonpickle.encode(response)
        return Response(response=response_pickled, status=200,
                        mimetype="application/json")

    image = np.asarray(bytearray(fileReceived[1]), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    with graph.as_default():
        res = reco.predict(image)

    # build a response dict to send back to client
    response = {'message': 'Your fruit is {}'.format(res)}

    response_pickled = jsonpickle.encode(response)

    return Response(response=response_pickled, status=200,
                    mimetype="application/json")


# start flask app
app.run(host='0.0.0.0', debug=False, threaded=False)

b'v0.4a-45-gd1621a6'
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Dec/2018 20:16:53] "POST /api/recognize HTTP/1.1" 200 -
